## Simple Example

  - Create two lists of `N` random numbers: `lst1` and `lst2`
  - Multiply each element `lst1[i]` to `lst2[i]` and place result in a new list `lst3`.
  - Remove all elements $<= 1.0$ from lst3 and put the result into lst4
  - Return the sum of square of all elements of `lst4`.


In [1]:
import random 
import timeit


def example1_imperative(N):
    lst1 = []
    lst2 = [] 
    lst3 = [] 
    lst4 = []
    sum_sq = 0.0
    for i in range(N):
        elt1 = random.uniform(-1.0, 1.0)
        lst1.append(elt1)
        elt2 = random.uniform(-1.0, 1.0)
        lst2.append(elt2)
        lst3.append(elt1 * elt2)
        if elt1 * elt2 > 1.0:
            lst4.append(elt4)
            sum_sq = sum_sq + elt4**2
    # .. further calculations with lst1,.., lst4
    return sum_sq 


## Simple Example: Made Functional 

In [2]:
def example1_functional(N):
    lst1 = [ random.uniform(-1.0, 1.0) for _ in range(N) ]
    lst2 = [ random.uniform(-1.0, 1.0) for _ in range(N) ]
    lst3 = [ elt1 + elt2 for (elt1, elt2) in zip(lst1, lst2) ]
    lst4 = [ e for e in lst3 if e > 1.0 ]
    return sum([x**2 for x in lst4]) 

## Functional Programming 

A technique for expressing computations, wherein:
  - Functions are first class objects that can be created and passed around.
  - Side-effects are avoided.
  - Avoid loops and mutations of data-structures.
  - ...


In [4]:
def factorial(n):
    f = 1
    for i in range(2, n+1):
        f = f * i
    return f 


In [5]:
def factorial_rec(n):
    if n <= 1:
        return 1
    else:
        return n * factorial_rec(n-1)

## Recursion versus Loops

Functional programming tries to replace loops with recursion.

**Problem:** Recursion is expensive!


**Lookup:** Tail vs. non-tail recursion.


In [6]:
def factorial_tail(n, res=1):
    if n <= 1:
        return res
    else:
        return factorial_tail(n-1, res*n)

## Lambda

- A `lambda` in python is a function.
- It can be passed to a function.
- It can be created from inside a function and passed outside.

In [14]:
lambda1 = lambda x,y: x + y

print(lambda1(10, 10))

lambda2 = lambda z: 10 + z 

print(lambda2(-10))

20
0


In [16]:
def map_fun(func, lst):
    return [func(elt) for elt in lst] 


In [17]:
map_fun(lambda2, list(range(10)))

[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [18]:
def create_a_lambda(j):
    return lambda x : j * x

In [22]:
lambda3  = create_a_lambda(10)
print(lambda3(10))
print(lambda3(-10))

100
-100


## Basic FP Concepts

 - Recursion, Tail Recursion, Replacing Loops with Recursion.
 - Lambdas (related notions: closure)
 - Functors: map, reduce, filter.
 - Inductive Data Types (if your language supports them).
 - Iterators: range, zip, ...

## Advanced FP Concepts

 - Monads
 - Lazy Programming
 - Continuations
 - Algebraic Effects 
 - Streams
 - Combinators
 - ... 


## Programming with Continuations

What is a continuation?

In [23]:
def factorial(n):
    if n <= 1:
        return 1
    else:
        # return n * factorial(n-1)
        v = factorial(n-1) 
        return v * n

**Continuation:** What should be done with the result.

 - If we called a function `factorial(n)` it returns `n!`.

In the _continuation passing style_, every function has an extra argument called a _continuation_.
  - The continuation is a lambda.
  - The function itself passes its result to the continuation.


In [24]:
def factorial_k(n, k):
    if n <= 1:
        return k(1)
    else:
        return factorial_k( n-1, lambda v: k(n*v) )

In [26]:
factorial_k(10, lambda x: print(f'Result: {x}'))

Result: 3628800


## Factorial with Continuations 

1. What is the meaning of this lambda?

` lambda v: k(n*v) ` 

2. What is the meaning of this recursive call?

` factorial_k( n-1, lambda v: k(n*v) ) `

Let's say it out loud in english.

**Interesting Note:** Any recursion can be converted into a tail recursion using continuations.

In [27]:
def collatz(n):
    if n <= 1:
        return 1
    elif n %2 == 0:
        return 1 + collatz(n//2)
    else:
        return 1 + collatz(3 * n + 1)

In [30]:
print(collatz(25))
print(collatz(127))

24
47


In [32]:
def collatz_cps(n, k):
    ## TODO during session.
    pass

In [34]:
collatz_cps(25, lambda x: print(x))
collatz_cps(127, lambda z: print(f'Result is {z}'))

## Trampolines 

Trampolines are a modification of continuations. 
  - Instead of calling the continuation, return it back to the caller.
  - What the trampoline returns is the _left over computation_ to be done when a result becomes available.



In [41]:
class More:
    def __init__(self, lamb):
        self.type = 'more'
        self.lamb = lamb

class Val:
    def __init__(self, v):
        self.type='done'
        self.value = v

In [63]:
def factorial_with_trampoline(n, k):
    print(f'In Trampoline: {n}')
    if n <= 1:
        return More( lambda: k(1) )
    else: 
        def remaining_computation(v):
            print(f' Doing remaining computation: v = {v}')
            return More( lambda: k(n*v) )
        return More (lambda: factorial_with_trampoline(n-1, remaining_computation) )

__What the...?__

In [66]:
def tram_factorial(n):
    terminal = lambda v: Val(v) 
    r = factorial_with_trampoline(n, terminal)
    while r.type != 'done':
        assert r.type == 'more'
        print(' In event loop') 
        l = r.lamb 
        r = l() 
    assert r.type == 'done'
    return r.value 


In [67]:
tram_factorial(10)

In Trampoline: 10
 In event loop
In Trampoline: 9
 In event loop
In Trampoline: 8
 In event loop
In Trampoline: 7
 In event loop
In Trampoline: 6
 In event loop
In Trampoline: 5
 In event loop
In Trampoline: 4
 In event loop
In Trampoline: 3
 In event loop
In Trampoline: 2
 In event loop
In Trampoline: 1
 In event loop
 Doing remaining computation: v = 1
 In event loop
 Doing remaining computation: v = 2
 In event loop
 Doing remaining computation: v = 6
 In event loop
 Doing remaining computation: v = 24
 In event loop
 Doing remaining computation: v = 120
 In event loop
 Doing remaining computation: v = 720
 In event loop
 Doing remaining computation: v = 5040
 In event loop
 Doing remaining computation: v = 40320
 In event loop
 Doing remaining computation: v = 362880
 In event loop


3628800

## What are trampolines?

- If you call a trampoline, they simply do _simple_ computation and send you back whatever remains to compute.
- In a trampolined code, there is simply a main "event" loop that dispatches calls to worker threads.
  
## How are they used?

- To implement exception handling.
- To implement multiple threads in a single threaded environment.
- Asynchronous calls/algebraic effects.
- **Event-Driven Programming**

## Reactive Programming Architecture